In [4]:
import csv
import praw
import datetime
from config import CLIENT_ID, CLIENT_SECRET, USERNAME, PASSWORD, USER_AGENT

# Initialize the Reddit API client. You need to create a token on Reddit to do this. see documentation.
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT,
)

print(reddit.read_only) # show whther or not i have the authenticated connection with the API

False


In [5]:
# Get your subreddit
subreddit = reddit.subreddit("covidvaccinated")

#Just doublechecking the info... 
print(subreddit.display_name)
# Output: redditdev
print(subreddit.title)
# Output: reddit development
print(subreddit.description)
# Output: a subreddit for discussion of ...

covidvaccinated
CovidVaccinated
Vaccines are coming ! Many types, from may manufacturers, at different times, for different people and in different places. Lets talk about vaccination and vaccines


In [6]:
# Get all comments from the subreddit
comments = list(subreddit.comments(limit=None))

# Open the CSV file for writing
csv_file = open('covidvaccinated_comments_praw.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Comment', 'Timestamp', 'Deleted', 'Comment Author', 'Distinguished', 'Nest Level', 'Total Comments', 'Comment Score'])

def write_comments(comments, nest_level, total_comments):
    for comment in comments:
        # Comment deleted?
        deleted = (comment.author is None)

        # Comment author
        comment_author = str(comment.author) if comment.author else None

        # Is Distinguished and if so, how?
        distinguished = comment.distinguished if comment.distinguished else ''

        # Convert the timestamp to a human-readable format!
        timestamp = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')

        # Write comment data to the CSV file
        csv_writer.writerow([comment.body, timestamp, deleted, comment_author, distinguished, nest_level, total_comments, comment.score])

        # Recursively write nested comments
        if len(comment.replies) > 0:
            write_comments(comment.replies, nest_level + 1, total_comments)

# Write comments and their nested replies
total_comments = len(comments)
write_comments(comments, 0, total_comments)

# Close the CSV file
csv_file.close()
